<a href="https://colab.research.google.com/github/kavsraj/Applied_Datascience_Capstone_Project/blob/master/Segmenting_%26_Clustering_Neighborhoods_in_Toronto_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####Import libraries

In [57]:
import pandas as pd
!pip -q install lxml

!pip install requests
import requests 
import io 

import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 
from folium import plugins

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


##### Web scrapping, Cleaning Data, Dataframe with the cleaned Data

In [62]:
#read the data from the url and set the index 
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"
df = pd.read_html(url)[0]

# Ignore cells with a borough that is 'Not assigned'.
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).to_frame().reset_index()

# Neighborhoods separated with a comma
for index,row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
      row['Neighbourhood'] = row['Borough']

#we need to get the latitude and the longitude coordinates of each neighborhood in order to utilize the Foursquare location data.
url1="http://cocl.us/Geospatial_data"
req=requests.get(url1).content
dec=pd.read_csv(io.StringIO(req.decode('utf-8')))

# rename the first column to allow merging dataframes on Postcode
dec.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(dec, df, on='Postcode')

# reorder column names and show the dataframe
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


##### dataframe with 'Borough' = 'Toronto

In [61]:
toronto_df = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#####Generating Map

 

In [0]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [64]:
#Show an initial map of the neighborhoods in Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto